In [ ]:
!pip install gradio
!pip install openai


In [ ]:
import openai

In [ ]:
def critique_prompt(prompt_text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert prompt engineer."},
                {"role": "user", "content": f"Critique the following prompt and suggest improvements: '{prompt_text}'"}
            ]
        )
        critique_full = response.choices[0].message['content'].strip()

        # Splitting the critique into suggestion and reason
        parts = critique_full.split('. ')
        if len(parts) > 1:
            suggested_prompt = parts[0]
            reason_for_suggestion = '. '.join(parts[1:])
        else:
            suggested_prompt = critique_full
            reason_for_suggestion = "Not specified"

        return prompt_text, suggested_prompt, reason_for_suggestion
    except Exception as e:
        return prompt_text, "Error", f"Error: {str(e)}"


In [ ]:
import gradio as gr

In [ ]:
def gradio_interface(original_prompt, refine_button):
    global input_history, suggestion_history, response_history

    if refine_button and suggestion_history:
        original_prompt = suggestion_history[-1]

    original, suggested_prompt, reason_for_suggestion = critique_prompt(original_prompt)

    input_history.append(original_prompt)
    suggestion_history.append(suggested_prompt)
    response_history.append('Refine')

    history_string = "\n".join([f"Input {i+1}: {inp}\nSuggestion {i+1}: {sug}\nResponse {i+1}: {res}\n"
                                for i, (inp, sug, res) in enumerate(zip(input_history, suggestion_history, response_history))])

    return f"Original Prompt: {original}", suggested_prompt, reason_for_suggestion, history_string

# Define the components of the interface
input_component = gr.Textbox(placeholder="Enter or refine your prompt here...", label="Your Prompt")
refine_button = gr.Button(label="Refine")

output_original = gr.Textbox(label="Original Prompt")
output_suggested_prompt = gr.Textbox(label="Suggested Prompt")
output_reason = gr.Textbox(label="Reason for Suggestion")
output_history = gr.Textbox(label="Interaction History", readonly=True)

app = gr.Interface(
    fn=gradio_interface,
    inputs=[input_component, refine_button],
    outputs=[output_original, output_suggested_prompt, output_reason, output_history],
    description="Refine your prompt using ChatGPT's best practices!",
    live=False  # This allows the user to see suggestions in real-time and iteratively refine the prompt
)
app.launch()

<ipython-input-33-a43a191078b7>:25: GradioUnusedKwargWarning: You have unused kwarg parameters in Textbox, please remove them: {'readonly': True}
  output_history = gr.Textbox(label="Interaction History", readonly=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://afcbd625d827f02e73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
